In [ ]:
from google.colab import drive
import sys
drive.mount('/content/drive')
sys.path.append('/content/drive/MyDrive/pcf-diffusion-mx_branch')

In [ ]:
from model.utils import timestep_embedding, timesteps_to_tensor
import torch.nn.functional as F

from diffusion import DiffusionProcess
import torch
from diffusion.utils.sde import SDE
import numpy as np
import logging

from pytorch_lightning import LightningDataModule
from tqdm import tqdm
from matplotlib import pyplot as plt
import seaborn as sns
from matplotlib import gridspec
import torch.nn as nn
from torch.distributions import Normal, Categorical, MixtureSameFamily
import torch
#device = 'mps'
device = 'cuda:0'
import random
from src.PCF_with_empirical_measure import PCF_with_empirical_measure

# SDE-based diffusion models with ToyNet generator

In [ ]:
class ResNet_FC(nn.Module):
    def __init__(self, data_dim, hidden_dim, num_res_blocks):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.map=nn.Linear(data_dim, hidden_dim)
        self.res_blocks = nn.ModuleList(
            [self.build_res_block() for _ in range(num_res_blocks)])

    def build_linear(self, in_features, out_features):
        linear = nn.Linear(in_features, out_features)
        return linear

    def build_res_block(self):
        hid = self.hidden_dim
        layers = []
        widths =[hid]*4  # [256, 256, 256, 256]
        for i in range(len(widths) - 1):
            layers.append(self.build_linear(widths[i], widths[i + 1]))
            layers.append(nn.SiLU())
        return nn.Sequential(*layers) # [batch_size, dim]

    def forward(self, x):  # x_t: [batch_size, data_dim]
        h=self.map(x)
        for res_block in self.res_blocks:
            h = (h + res_block(h)) / np.sqrt(2)
        return h   # [batch_size, dim]

In [ ]:
class ToyNet(nn.Module):
    def __init__(self, data_dim, direction=None):
        super(ToyNet, self).__init__()
        self.direction = direction

        self.time_embed_dim = 128
        dim = 256
        out_dim = data_dim

        self.t_module = nn.Sequential(nn.Linear(self.time_embed_dim, dim), nn.SiLU(), nn.Linear(dim, dim),)
        self.x_module = ResNet_FC(data_dim, dim, num_res_blocks=1)
        self.out_module = nn.Sequential(nn.Linear(dim,dim), nn.SiLU(), nn.Linear(dim, out_dim),)
        
    def forward(self, x, t: int or list[int]): # x_t: [batch_size, data_dim]
        t = timesteps_to_tensor(t, batch_size=x.shape[0]).to(x.device)
        t_emb = timestep_embedding(t, self.time_embed_dim).to(x.device)  # [batch_size, time_embed_dim]

        t_out = self.t_module(t_emb) # [batch_size, dim=256]
        x_out = self.x_module(x) # [batch_size, dim=256]
        out   = self.out_module(x_out+t_out) # [batch_size, out_dim=data_dim]

        return out

In [ ]:
class SDEProcess(DiffusionProcess):
    def __init__(self, sde_type="VP",
                 sde_info={"VP": {"beta_min": 0.1, "beta_max": 20},
                           "subVP": {"beta_min": 0.1, "beta_max": 20},
                           "VE": {"sigma_min": 0.01, "sigma_max": 50}}, 
                 **kwargs):
        super(SDEProcess, self).__init__(**kwargs)
        assert self.discrete is False, "DDPM is only for continuous data"
        self.dt = 1. / self.total_steps # step size
        self.sde = SDE(self.total_steps, sde_type, sde_info)
        
    def forward_one_step(self, x_prev, t):
        """
        Discretized forward SDE process for actual compuatation: 
        x_{t+1} = x_t + f_t(x_t) * dt + G_t * z_t * sqrt(dt)
        """
        f_t, g_t = self.sde.drifts(x_prev, t-1)
        z = torch.randn_like(x_prev)
        x_t = x_prev + f_t * self.dt + g_t * z * np.sqrt(self.dt)
        return x_t # [N]

    
    def backward_one_step(self, x_t, t, pred_score, clip_denoised=True):
        """
        Discretized backward SDE process for actual compuatation:
        x_{t-1} = x_t - (f_t(x_t) - (G_t)^2 * pred_score) * dt + G_t * z_t * sqrt(dt)
        """
        z = torch.randn_like(x_t).to(device) # [N,d=1]
        f_t, g_t = self.sde.drifts(x_t, t)
        f_t = f_t.to(device)
        x_prev = x_t - (f_t - g_t**2 * pred_score) * self.dt + g_t * z * np.sqrt(self.dt)
        if clip_denoised and x_t.ndim > 2:
            print('backward_one_step')
            x_prev.clamp_(-1., 1.) 

        return x_prev # [N,d=1]

    @torch.no_grad()
    def sample(self, noise, net):
        """
        Sample from backward diffusion process
        """
        x_t = noise # [N,1]
        trajs = [x_t]

        for t in reversed(range(1, self.total_steps+1)):
            pred_score = net(x_t, t)
            x_t = self.backward_one_step(x_t, t, pred_score)
            trajs.append(x_t)
        return x_t, torch.hstack(trajs)
    
    def forward_sample(self, data): # data: torch.tensor, shape=[N]
        trajs = torch.zeros([len(data), self.total_steps+1]) # [N, T+1]
        x = data.to(device) # [N]
        trajs[:, 0] = x
        for t in range(1, self.total_steps+1):
            x = self.forward_one_step(x, t) # [N]
            trajs[:, t] = x
        return x, trajs # [N], [N, T+1]
    
    def backward_sample(self, noise, net):
        """
        Sample from backward diffusion process
        """
        x_t = noise # [N,d=1]
        trajs = [x_t]

        for t in reversed(range(1, self.total_steps+1)):
            pred_score = net(x_t, t)
            x_t = self.backward_one_step(x_t, t, pred_score)
            trajs.append(x_t)
        return x_t, torch.hstack(trajs) # [N,d=1], [N,(T+1)*d]
        
    def backward_sample_example(self, noise, net):
        """
        Sample from backward diffusion process
        noise: torch.tensor, shape=[num_paths, T]
        """
        # noise_start = noise[:, 0]
        noise_end = noise[:, -1]
        # trajs = [noise_end]
        trajs = torch.zeros_like(noise)
        trajs[:, -1] = noise_end
        for t in reversed(range(1, self.total_steps+1)):
            pred_score = net(noise[:,t].unsqueeze(1), t) # [N,d=1]
            x_t = self.backward_one_step(noise[:,t].unsqueeze(1), t, pred_score)
            trajs[:,t-1] = x_t.squeeze()
        return x_t, trajs


In [ ]:
# Data distribution
mix = Categorical(torch.ones(2,))
comp = Normal(torch.tensor([0 - 2., 0 + 2.]).to(device), torch.tensor([.5, .5]).to(device))
data_dist = MixtureSameFamily(mix, comp)

num_paths = 32
print(type(data_dist))  # <class 'torch.distributions.mixture_same_family.MixtureSameFamily'>
print(data_dist.batch_shape)  # torch.Size([2])
print(data_dist.event_shape)  # torch.Size([])


In [ ]:
score_net = ToyNet(data_dim=1).to(device)
total_steps = 100
num_paths = 32
diffusion = SDEProcess(discrete=False, total_steps=total_steps, sde_type="VP")

In [ ]:
def loss_function(x0, diffusion_process, score_net, t):
    #f_t is not involved in the score matching loss soe input zeros for the 1st argument.
    #
    _, g_t = diffusion_process.sde.drifts(torch.zeros_like(x0), t)
    
    # Eq 29 in Score based diffusion model (Yang Song 2021 ICLR)
    mean, std = diffusion_process.sde.perturbation_kernel(x0, t)
    z = torch.randn_like(x0)

    x_t = mean + std * z  # x_t is perturbed x_0
    pred_score = score_net(x_t, t)
    target = -z / std   # Since desired derivative of the given kernel is - (x_t - mean) / std^2
    loss = g_t**2 * F.mse_loss(pred_score, target, reduction="sum") / np.prod(x0.size())
    return loss

In [ ]:
x_data = data_dist.sample([num_paths]).to(device)
x_noise, traj_forward = diffusion.forward_sample(x_data)

In [ ]:
num_epochs = 2000
opt = torch.optim.Adam(score_net.parameters(), lr=1e-3)

for i in range(num_epochs):
    # Forward process to generate time series
    opt.zero_grad()
    x0 = data_dist.sample([num_paths]).to(device)
    t = np.random.randint(1, total_steps + 1)
    loss = loss_function(x0.view(-1 ,1), diffusion, score_net, t)
    loss.backward()
    opt.step()
    if(i%50==0):
        x0 = data_dist.sample([num_paths]).to(device)
        t = np.random.randint(1, total_steps + 1)
        loss = loss_function(x0.view(-1 ,1), diffusion, score_net, t)
        loss.backward()
        opt.step()
    if(i%50==0):
        print(i, loss)

In [ ]:
xxx = np.linspace(0, 1, traj_forward.shape[1])

plt.figure(figsize=(12,3))

plt.subplot(1, 2, 1)
for line in traj_forward:
    plt.plot(xxx, line.cpu(), linewidth=1.0)
    plt.title('forward')
plt.ylim(-5, 5)
    
plt.subplot(1, 2, 2)
x_gen, traj_back = diffusion.sample(torch.randn([num_paths, 1]).to(device), score_net)
for line in traj_back:
    plt.plot(xxx, line.cpu(), linewidth=1.0)
    plt.title('backward')
plt.ylim(-5, 5)

plt.show()

# local PCFD as loss function as ToyNet as generator

In [ ]:
pcf = PCF_with_empirical_measure(8, 5, 1, add_time=True)
pcf = pcf.to(device)
num_paths = 64
G_opt = torch.optim.Adam(score_net.parameters(), lr=5e-4, betas=(0.9, 0.999))
D_opt = torch.optim.Adam(pcf.parameters(), lr=5e-3, betas=(0.9, 0.999))
n_Dstep_per_Gstep = 10
n_Gstep_per_Dstep = 1
t_max_step = 80

num_epochs = 5
ts_length = 20
forecast_length = 20

for i in range(num_epochs):
    x_data = data_dist.sample([num_paths]).to(device) # [N]
    x_noise, traj_forward = diffusion.forward_sample(x_data) # [N, total_steps+1]
    traj_forward = traj_forward.to(device) # [N, total_steps+1]
    traj_forward_flip = torch.fliplr(traj_forward)
             
    # train the PCF network as a discriminator. We want PCF has a great ablitily to distinguish the real data and the generated data. 
    # So we train the PCF to maxmise the distance between the real data and the generated data.
   
        #x_gen, traj_back = diffusion.backward_sample(torch.randn([num_paths, 1]).to(device), score_net)
        #x_gen, traj_back = diffusion.backward_sample(x_noise.to(device), score_net)

    for j in range(n_Gstep_per_Dstep):  
        with torch.no_grad():
            x_gen, traj_back = diffusion.backward_sample_example(traj_forward.to(device), score_net)
            t = torch.randint(t_max_step, (ts_length,)).to(device)    
    
            x_real=traj_forward_flip.view([-1,total_steps+1, 1]) # [N, total_steps+1, 1]
            x_fake = traj_back.view([-1, total_steps+1, 1]) # [N, total_steps+1, 1]
        G_loss = 0
        for t1 in range(ts_length):
            G_loss = G_loss + pcf.distance_measure(x_real[:, [t1, t1+forecast_length]], 
                                x_fake[:, [t1,  t1+forecast_length]], Lambda=0.1) 
        G_loss = G_loss/ts_length
        G_opt.zero_grad()
        G_loss.backward()
        torch.nn.utils.clip_grad_norm_(score_net.parameters(), 0.05)

        G_opt.step()
        pcf.eval()  # Set PCF model to evaluation mode
        print(j,  G_loss.item())
        
    for j in range(n_Dstep_per_Gstep):
        with torch.no_grad():
            x_data = data_dist.sample([num_paths]).to(device)
            x_noise, traj_forward = diffusion.forward_sample(x_data)
            traj_forward = traj_forward.to(device)
            x_gen, traj_back = diffusion.backward_sample(torch.randn([num_paths, 1]).to(device), score_net) 
        traj_forward_flip = torch.fliplr(traj_forward)
       # G_loss =  pcf.distance_measure(traj_forward_flip.view([-1, total_steps+1, 1]), 
                               # traj_back.view([-1, total_steps+1, 1]), Lambda=0) 
           
        x_real=traj_forward_flip.view([-1,total_steps+1, 1])
        x_fake = traj_back.view([-1, total_steps+1, 1])
        pcf.train()  # Set PCF model to training mode
    #D_loss = -pcf.distance_measure(traj_forward_flip.view([-1,  total_steps+1, 1]), 
                               # traj_back.view([-1, total_steps+1, 1]), Lambda=0.0) 
        t = torch.randint(t_max_step, (ts_length,)).to(device) 
        D_loss = 0
        for t1 in range(ts_length):
            D_loss = D_loss - pcf.distance_measure(x_real[:, [t1, t1+forecast_length]], 
                                x_fake[:, [t1,  t1+forecast_length]], Lambda=0.1) 
        D_loss =  D_loss/ts_length

        D_opt.zero_grad()
        D_loss.backward()
        D_opt.step()
        pcf.eval()  # Set PCF model to evaluation mode
        print(j, D_loss.item())
    


# local PCFD as loss function and SigRNN as generator

In [ ]:
class FFN(nn.Module):

    def __init__(self, input_dim: int, output_dim: int, hidden_dims: Tuple[int]):
      super().__init__()      
      blocks = []
      input_dim_block = input_dim

      for hidden_dim in hidden_dims:
          blocks.append(nn.Linear(input_dim_block, hidden_dim))
          blocks.append(nn.PReLU())
          input_dim_block = hidden_dim
      blocks.append(nn.Linear(input_dim_block, output_dim))
      self.network = nn.Sequential(*blocks)
      self.output_dim = output_dim

    def forward(self, *args):
      x = torch.cat(args, -1)
      out = self.network(x)
      return out

class ResidualBlock(nn.Module):
    def __init__(self, input_dim: int, output_dim: int):
        super(ResidualBlock, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)
        self.activation = nn.ReLU()
        self.create_residual_connection = True if input_dim == output_dim else False

    def forward(self, x):
        y = self.activation(self.linear(x))
        if self.create_residual_connection:
            y = x + y
        return y

class ResFNN(nn.Module):
    def __init__(self, input_dim: int, output_dim: int, hidden_dims: Tuple[int], flatten: bool = False):
        """
        Feedforward neural network with residual connection.
        Args:
            input_dim: integer, specifies input dimension of the neural network
            output_dim: integer, specifies output dimension of the neural network
            hidden_dims: list of integers, specifies the hidden dimensions of each layer.
                in above definition L = len(hidden_dims) since the last hidden layer is followed by an output layer
        """
        super(ResFNN, self).__init__()
        blocks = list()
        self.input_dim = input_dim
        self.flatten = flatten
        input_dim_block = input_dim
        for hidden_dim in hidden_dims:
            blocks.append(ResidualBlock(input_dim_block, hidden_dim))
            input_dim_block = hidden_dim
        blocks.append(nn.Linear(input_dim_block, output_dim))
        self.network = nn.Sequential(*blocks)
        self.blocks = blocks

    def forward(self, x):
        if self.flatten:
            x = x.reshape(x.shape[0], -1)
        out = self.network(x)
        return out



In [ ]:
class OneStepGenerator(nn.Module):
  def __init__(self, input_dim, hidden_dim, num_layers, output_dim=1):
    super(OneStepGenerator, self).__init__()

    self.input_dim = input_dim
    self.hidden_dim = hidden_dim
    self.num_layers = num_layers
    self.output_dim = output_dim
    self.time_embed_dim=128

    # self.rnn = nn.Sequential(FFN(input_dim = hidden_dim+1+2*input_dim,
    #                               output_dim = hidden_dim,
    #                               hidden_dims = [hidden_dim, hidden_dim]),  # Two hidden layers, each with hidden_dim units
    #                               nn.SiLU()
    #                         )
    # self.linear = nn.Linear(in_features=hidden_dim, out_features=output_dim)
    # self.initial_nn = nn.Sequential(ResFNN(input_dim=input_dim,
    #                                         output_dim=hidden_dim,
    #                                         hidden_dims=[hidden_dim, hidden_dim]),
    #                                 nn.SiLU()
    #                                 )
    
    self.t_module = nn.Sequential(nn.Linear(self.time_embed_dim, self.hidden_dim), nn.SiLU(), nn.Linear(self.hidden_dim, self.hidden_dim),)
    self.eps_module = ResNet_FC(2*input_dim, hidden_dim, num_res_blocks=1)
    self.x_module = nn.Sequential(ResFNN(input_dim=input_dim, output_dim=hidden_dim, hidden_dims=[hidden_dim, hidden_dim]), nn.SiLU())
    self.out_module = nn.Sequential(nn.Linear(hidden_dim,hidden_dim), nn.SiLU(), nn.Linear(hidden_dim, output_dim),)

  def forward(self, x_t, t, eps): # noise: [N,d=1]
    t = timesteps_to_tensor(t, batch_size=x_t.shape[0]).to(x_t.device)
    t_emb = timestep_embedding(t, self.time_embed_dim).to(x_t.device)  # [batch_size, time_embed_dim]
    t_out = self.t_module(t_emb).to(x_t.device)  # [batch_size, hidden_dim]
    eps_out = self.eps_module(eps).to(x_t.device) # [batch_size, hidden_dim]
    x_out = self.x_module(x_t).to(x_t.device) # [batch_size, hidden_dim]
    out = self.out_module(x_out + t_out + eps_out).to(x_t.device) # [batch_size, out_dim=data_dim]
    return out # [N,output_dim=1]



In [ ]:
class SigRNNDiffusion(DiffusionProcess):
    def __init__(self, sde_type="VP",
                 sde_info={"VP": {"beta_min": 0.1, "beta_max": 20},
                           "subVP": {"beta_min": 0.1, "beta_max": 20},
                           "VE": {"sigma_min": 0.01, "sigma_max": 50}},
                 c_total_steps = 20,
                 **kwargs):
        super(SigRNNDiffusion, self).__init__(**kwargs)
        assert self.discrete is False, "DDPM is only for continuous data"
        self.c_total_steps = c_total_steps
        self.dt = 1. / self.total_steps # step size
        self.c_dt = 1. / self.c_total_steps
        self.sde = SDE(self.total_steps, sde_type, sde_info)

    def forward_one_step(self, x_prev, t):
        """
        Discretized forward SDE process for actual compuatation:
        x_{t+1} = x_t + f_t(x_t) * dt + G_t * z_t * sqrt(dt)
        """
        f_t, g_t = self.sde.drifts(x_prev, t-1)
        z = torch.randn_like(x_prev)
        x_t = x_prev + f_t * self.dt + g_t * z * np.sqrt(self.dt)
        return x_t # [N]

    def forward_sample(self, data): # data: torch.tensor, shape=[N]
        trajs = torch.zeros([len(data), self.total_steps+1]) # [N,T+1]
        x = data.to(device) # [N]
        trajs[:, 0] = x
        for t in range(1, self.total_steps+1):
            x = self.forward_one_step(x, t) # [N]
            trajs[:, t] = x
        return x, trajs # [N]:the last step, [N,T+1]

    def sample_eps(self, batch_size, input_dim, c_dt):
      # generate i.i.d. d=input_dim joint Gaussian rvs with defined covariance matrix
      eps = []
      for _ in range(input_dim):
        cov_matrix = torch.tensor([[c_dt, (c_dt**2/2)],
                                [(c_dt**2/2), (c_dt**3/3)]])
        m = MultivariateNormal(loc=torch.zeros(2), covariance_matrix=cov_matrix)
        eps_i = m.sample((batch_size,))
        eps.append(eps_i)
      eps = torch.cat(eps, dim=1) # [N, 2*input_dim]
      return eps.to(device)  # [N, 2*input_dim]

    def backward_one_step(self, x_t, t, generator, clip_denoised=True):
        eps = self.sample_eps(batch_size=x_t.shape[0], input_dim=x_t.shape[1], c_dt=self.c_dt).to(x_t.device) # [N,2*d=2]
        t_tensor = torch.tensor([t], device=x_t.device).unsqueeze(0).repeat(x_t.shape[0], 1) # [N,1]
        x_prev = generator(x_t, t_tensor, eps).to(x_t.device)
        if clip_denoised and x_t.ndim > 2:
            print('backward_one_step')
            x_prev.clamp_(-1., 1.)
        return x_prev # [N,d=1]

    @torch.no_grad()
    def sample(self, noise, generator):
        """
        Sample from backward diffusion process
        """
        # x_t = noise.to(device) # [N,d=1]
        x_t = noise # [N,d=1]
        trajs = [x_t]
        time_c = torch.linspace(self.c_dt,1.,self.c_total_steps)
        for t in reversed(time_c):
          x_prev = self.backward_one_step(x_t,t,generator) # [N,d=1]
          trajs.append(x_prev)
        return x_t, torch.hstack(trajs) # [N,d=1], [N,(c_T+1)*(d=1)]

    def backward_sample(self, noise, generator):
        """
        Sample from backward diffusion process
        """
        # x_t = noise.to(device) # [N,d=1]
        x_t = noise # [N,d=1]
        trajs = [x_t]
        time_c = torch.linspace(self.c_dt,1.,self.c_total_steps)
        for t in reversed(time_c):
          x_prev = self.backward_one_step(x_t,t,generator) # [N,d=1]
          trajs.append(x_prev)
        return x_t, torch.hstack(trajs) # [N,d=1], [N,(c_T+1)*(d=1)]

    def backward_sample_example(self, noise, generator):
        """
        Sample from backward diffusion process
        noise: torch.tensor, shape=[num_paths, T]
        """
        noise_end = noise[:, -1].to(device) #[N]
        trajs = torch.zeros_like(noise).to(device) #[N,T]
        trajs[:, -1] = noise_end
        time_incre = torch.linspace(self.c_dt, 1., self.c_total_steps).to(device)
        time_index = torch.linspace(1, self.c_total_steps, self.c_total_steps).to(device)
        for idx, t in reversed(list(zip(time_index, time_incre))):
            x_t = self.backward_one_step(noise[:, int(idx)].unsqueeze(1).to(device), t, generator) # [N,d=1]
            trajs[:, int(idx)-1] = x_t.squeeze()
        return x_t, trajs # [N,d=1], [N,T]



In [ ]:
total_steps = 20
c_total_steps = 20

pcf = PCF_with_empirical_measure(8, 5, 1, add_time=True)  # PCFD
pcf = pcf.to(device)

# generator = sigRNNGenerator(input_dim=1, hidden_dim=128, num_layers=1, output_dim=1)
generator = OneStepGenerator(input_dim=1, hidden_dim=128, num_layers=1, output_dim=1).to(device)
generator = generator.to(device)
diffusion_sigrnn = SigRNNDiffusion(discrete=False, total_steps=total_steps, sde_type="VP", c_total_steps=c_total_steps)

batch_size = 32
num_epochs = 5
G_opt = torch.optim.Adam(generator.parameters(), lr=5e-4, betas=(0.9, 0.999))
D_opt = torch.optim.Adam(pcf.parameters(), lr=5e-3, betas=(0.9, 0.999))
n_Dstep_per_Gstep = 5
n_Gstep_per_Dstep = 5

t_max_step = 15
c_ts_length = 5
c_forecast_length = 5

for i in range(num_epochs):
  x_data = data_dist.sample([batch_size])
  x_noise, traj_forward = diffusion_sigrnn.forward_sample(x_data)
  traj_forward = traj_forward.to(device)
  traj_forward_flip = torch.fliplr(traj_forward)

  for j in range(n_Gstep_per_Dstep):
    with torch.no_grad():
      noise = torch.randn([batch_size,1]).to(device) # [N, d=1]
      # x_gen, traj_back = diffusion_sigrnn.backward_sample(x_noise.to(device), generator)
      x_gen, traj_back = diffusion_sigrnn.backward_sample_example(traj_forward.to(device), generator)
      traj_back = traj_back.to(device)
      ts = torch.randint(t_max_step, (c_ts_length,))
      x_real=traj_forward_flip.view([-1,total_steps+1, 1])
      x_fake = traj_back.view([-1, c_total_steps+1, 1])

    G_loss = 0
    for t1 in ts:
      ratio = total_steps / c_total_steps
      t1_real = int(t1*ratio)
      t1_real_forecast = int((t1+c_forecast_length)*ratio)
      # print(f't1_real={t1_real}, t1_real_forecast={t1_real_forecast}')
      # print(f't1={t1_real}, t1+c_forecast_length={t1+c_forecast_length}')
      G_loss = G_loss + pcf.distance_measure(x_real[:, t1_real:t1_real_forecast+1],
                                             x_fake[:, t1:t1+c_forecast_length+1], Lambda=0.1)
    G_loss = G_loss/ts_length
    G_opt.zero_grad()
    G_loss.backward()
    torch.nn.utils.clip_grad_norm_(generator.parameters(), 0.05)
    G_opt.step()
    pcf.eval()  # set PCF model to evaluation mode
    print(j,  G_loss.item())

  # train the PCF network as a discriminator. We want PCF has a great ability to distinguish the real data and the generated data.
  # so we train the PCF to maximise the distance between the real data and the generated data.
  for j in range(n_Dstep_per_Gstep):
    with torch.no_grad():
      x_data = data_dist.sample([batch_size])
      x_noise, traj_forward = diffusion_sigrnn.forward_sample(x_data)
      traj_forward = traj_forward.to(device)
      noise = torch.randn([batch_size, 1]).to(device)
      x_gen, traj_back = diffusion_sigrnn.backward_sample(noise, generator)
      traj_back = traj_back.to(device)
    traj_forward_flip = torch.fliplr(traj_forward)
    x_real=traj_forward_flip.view([-1,total_steps+1, 1])
    x_fake = traj_back.view([-1, c_total_steps+1, 1])

    pcf.train()  # set PCF model to training mode
    ts = torch.randint(t_max_step, (c_ts_length,))
    D_loss = 0
    for t1 in ts:
      ratio = total_steps / c_total_steps
      t1_real = int(t1 * ratio)
      t1_real_forecast = int((t1+c_forecast_length)* ratio)
      D_loss = D_loss - pcf.distance_measure(x_real[:, t1_real:t1_real_forecast+1],
                                             x_fake[:, t1:t1+c_forecast_length+1], Lambda=0.1)
    D_loss = D_loss/ts_length
    D_opt.zero_grad()
    D_loss.backward()
    D_opt.step()

    pcf.eval()  # set PCF model to evaluation mode
    print(j, D_loss.item())



In [ ]:
# poor performance
class sigRNNGenerator(nn.Module):
  def __init__(self, input_dim, hidden_dim, num_layers, output_dim=1):
    super(sigRNNGenerator, self).__init__()
    self.input_dim = input_dim
    self.hidden_dim = hidden_dim
    self.num_layers = num_layers
    self.output_dim = output_dim
    self.rnn = nn.Sequential(FFN(input_dim = hidden_dim+1+2*input_dim,
                                  output_dim = hidden_dim,
                                  hidden_dims = [hidden_dim, hidden_dim]),  # Two hidden layers, each with hidden_dim units
                                  nn.SiLU()
                            )
    self.linear = nn.Linear(in_features=hidden_dim, out_features=output_dim)
    self.initial_nn = nn.Sequential(ResFNN(input_dim=input_dim,
                                            output_dim=hidden_dim,
                                            hidden_dims=[hidden_dim, hidden_dim]),
                                    nn.SiLU()
                                    )

  def forward(self, batch_size, c_total_steps, c_dt, noise): # noise: [N,d=1]
    h = self.initial_nn(noise) # [N, hidden_dim]
    x = torch.zeros(batch_size, c_total_steps, self.output_dim)  # [N,c_T,output_dim=1]
    c_times = torch.linspace(c_dt,1,c_total_steps)

    for idx,t in enumerate(reversed(c_times)):
      eps = self.sample_eps(batch_size, c_dt) # [N, 2*d=2]
      h = self.rnn(torch.cat([h, torch.tensor([t]).expand(batch_size,1), eps], -1))  # [N, hid+1+2*in]
      x[:,idx,:] = self.linear(h)   # [N,output_dim=1]
    return x # [N,c_T,output_dim=1]

  def sample_eps(self, batch_size, c_dt):
    # generate i.i.d. d=input_dim joint Gaussian rvs with defined covariance matrix
    eps = []
    for _ in range(self.input_dim):
      cov_matrix = torch.tensor([[c_dt, (c_dt**2/2)],
                                 [(c_dt**2/2), (c_dt**3/3)]])
      m = MultivariateNormal(loc=torch.zeros(2), covariance_matrix=cov_matrix)
      eps_i = m.sample((batch_size,))
      eps.append(eps_i)
    eps = torch.cat(eps, dim=1) # [N, 2*input_dim]
    return eps


In [ ]:
# poor performance
class RNNGenerator(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim=1):
      super(RNNGenerator, self).__init__()
      self.input_dim = input_dim
      self.hidden_dim = hidden_dim
      self.num_layers = num_layers
      self.output_dim = output_dim
      self.linear = nn.Linear(in_features=hidden_dim, out_features=output_dim)
      self.initial_nn = nn.Sequential(ResFNN(input_dim=input_dim,
                                             output_dim=hidden_dim,
                                             hidden_dims=[hidden_dim, hidden_dim]),
                                      nn.SiLU()
                                      )
      self.rnn = nn.RNN(input_size=2*input_dim,hidden_size=hidden_dim,num_layers=num_layers,batch_first=True)

    def forward(self, batch_size, c_total_steps, c_dt, noise): # noise: [N,d=1]
      h = torch.randn(self.num_layers, batch_size, self.rnn.hidden_size)# [N, hidden_dim]
      input_seq = self.sample_eps(batch_size, c_total_steps, c_dt) # [N, c_T, 2*input_dim]
      out,_ = self.rnn(input_seq,h)
      x = self.linear(out)
      return x # [N,c_T,output_dim=1]

    def sample_eps(self, batch_size, c_total_steps, c_dt):
      # generate i.i.d. d=input_dim joint Gaussian rvs with defined covariance matrix
      eps = [] # [N, c_T, 2*input_dim]
      print(f"input_dim: {self.input_dim}, c_total_steps: {c_total_steps}, c_dt: {c_dt}")
      for t in range(c_total_steps):
        eps_t = []  # [N,1,2*input_dim]
        for _ in range(self.input_dim):
          cov_matrix = torch.tensor([[c_dt, (c_dt**2/2)],
                                  [(c_dt**2/2), (c_dt**3/3)]])
          m = MultivariateNormal(loc=torch.zeros(2), covariance_matrix=cov_matrix)
          eps_i = m.sample((batch_size,)) # [N,2]
          eps_t.append(eps_i)
        eps_t = torch.cat(eps_t, dim=1).unsqueeze(1) # [N,2*input_dim] -> [N,1,2*input_dim]
        eps.append(eps_t) 
      eps = torch.cat(eps,dim=1)
      print(f"eps shape: {eps.shape}")
      return eps # [N, c_T, 2*input_dim]



In [ ]:
# poor performance
class SigRNNDiffusion(DiffusionProcess):
    def __init__(self, sde_type="VP",
                 sde_info={"VP": {"beta_min": 0.1, "beta_max": 20},
                           "subVP": {"beta_min": 0.1, "beta_max": 20},
                           "VE": {"sigma_min": 0.01, "sigma_max": 50}},
                 c_total_steps = 20,
                 **kwargs):
        super(SigRNNDiffusion, self).__init__(**kwargs)
        assert self.discrete is False, "DDPM is only for continuous data"
        self.c_total_steps = c_total_steps
        self.dt = 1. / self.total_steps # step size
        self.c_dt = 1. / self.c_total_steps
        self.sde = SDE(self.total_steps, sde_type, sde_info)

    def forward_one_step(self, x_prev, t):
        """
        Discretized forward SDE process for actual compuatation:
        x_{t+1} = x_t + f_t(x_t) * dt + G_t * z_t * sqrt(dt)
        """
        f_t, g_t = self.sde.drifts(x_prev, t-1)
        z = torch.randn_like(x_prev)
        x_t = x_prev + f_t * self.dt + g_t * z * np.sqrt(self.dt)
        return x_t # [N]

    def forward_sample(self, data): # data: torch.tensor, shape=[N]
        trajs = torch.zeros([len(data), self.total_steps+1]) # [N,T+1]
        x = data.to(device) # [N]
        trajs[:, 0] = x
        for t in range(1, self.total_steps+1):
            x = self.forward_one_step(x, t) # [N]
            trajs[:, t] = x
        return x, trajs # [N]:the last step, [N,T+1]

    # def backward_one_step(self, x_t, t, pred_score, clip_denoised=True):
    #     """
    #     Discretized backward SDE process for actual compuatation:
    #     x_{t-1} = x_t - (f_t(x_t) - (G_t)^2 * pred_score) * dt + G_t * z_t * sqrt(dt)
    #     """
    #     z = torch.randn_like(x_t).to(device) # [N,d=1]
    #     f_t, g_t = self.sde.drifts(x_t, t)
    #     f_t = f_t.to(device)
    #     x_prev = x_t - (f_t - g_t**2 * pred_score) * self.dt + g_t * z * np.sqrt(self.dt)
    #     if clip_denoised and x_t.ndim > 2:
    #         print('backward_one_step')
    #         x_prev.clamp_(-1., 1.)
    #     return x_prev # [N,d=1]

    @torch.no_grad()
    def sample(self, noise, generator):
        """
        Sample from backward diffusion process
        """
        x_t = noise.to(generator.linear.weight.device) # [N,d=1]
        pred_trajs = generator(batch_size=x_t.shape[0], c_total_steps=self.c_total_steps, c_dt=self.c_dt, noise=x_t) # [N,c_T,d=1]
        pred_trajs = torch.reshape(torch.cat((x_t.unsqueeze(1), pred_trajs), dim=1),(x_t.shape[0],-1))  # [N,c_T+1,d=1] -> [N,(c_T+1)*(d=1)]
        x_gen = pred_trajs[:,-1] # [N,d=1]
        return x_gen, torch.reshape(pred_trajs,(x_t.shape[0],-1)) # [N,d=1], [N,(c_T+1)*(d=1)]

    def backward_sample(self, noise, generator):
        """
        Sample from backward diffusion process
        """
        x_t = noise.to(generator.linear.weight.device) # [N,d=1]
        pred_trajs = generator(batch_size=x_t.shape[0], c_total_steps=self.c_total_steps, c_dt=self.c_dt, noise=x_t) # [N,c_T,d=1]
        pred_trajs = torch.reshape(torch.cat((x_t.unsqueeze(1), pred_trajs), dim=1),(x_t.shape[0],-1))  # [N,c_T+1,d=1] -> [N,(c_T+1)*(d=1)]
        x_gen = pred_trajs[:,-1] # [N,d=1]
        return x_gen, torch.reshape(pred_trajs,(x_t.shape[0],-1)) # [N,d=1], [N,(c_T+1)*(d=1)]

    # def backward_sample_example(self, noise, net):
    #     """
    #     Sample from backward diffusion process
    #     noise: torch.tensor, shape=[num_paths, T]
    #     """
    #     # noise_start = noise[:, 0]
    #     noise_end = noise[:, -1]
    #     # trajs = [noise_end]
    #     trajs = torch.zeros_like(noise)
    #     trajs[:, -1] = noise_end
    #     for t in reversed(range(1, self.total_steps+1)):
    #         pred_score = net(noise[:,t].unsqueeze(1), t) # [N,d=1]
    #         x_t = self.backward_one_step(noise[:,t].unsqueeze(1), t, pred_score)
    #         trajs[:,t-1] = x_t.squeeze()
    #     return x_t, trajs

